In [1]:
import gspread
from dotenv import load_dotenv
from google.oauth2.service_account import Credentials
from mysql import connector
import pandas as pd
import os
import numpy as np
import mysql

In [2]:
load_dotenv()

True

In [3]:
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

In [4]:
creds = Credentials.from_service_account_file(
    '../' + os.getenv('CREDENTIALS_PATH'),
    scopes=SCOPES
)

In [5]:
gc = gspread.authorize(creds)

In [6]:
sheet = gc.open(os.getenv('SHEET_APPLICATION_NAME')).sheet1

In [7]:
row_values = sheet.get_all_records()

In [8]:
df = pd.DataFrame(row_values)

In [46]:
db = connector.connect(
    host=os.getenv('DB_IP'),
    port=os.getenv('DB_PORT'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME'),
)

In [47]:
cursor = db.cursor()

In [48]:
def null_prevention(raw_answer:str) -> str:
    raw_answer = str(raw_answer)
    if not raw_answer.strip() == '':
        return raw_answer
    else:
        return None

In [49]:
cursor.execute('SELECT student_id FROM students')

In [50]:
existing_student_ids = set(row[0] for row in cursor.fetchall())

In [51]:
existing_student_ids

{20221502065, 20222502096}

In [52]:
row_values[0].keys()

dict_keys(['Timestamp', 'First Name', 'Last Name', 'University ID', 'University Email Address', 'Phone Number', 'What is your faculty?', 'What is your major? (eg. Computer Engineering)', 'What year are you in?', 'How familiar are you with AI', 'Credit Hours Passed', 'LinkedIn (Optional)', 'GitHub (Optional)', 'Resume (Optional but encouraged)', 'Which activity are you likely to join', 'Gender', 'Which community are you a part of? (Optional)'])

In [53]:
row_values[1]['Which activity are you likely to join']

'Carnavals, Hackathons, Programming Competitions, Internships'

In [54]:
existing_student_ids

{20221502065, 20222502096}

In [55]:
int(row['University ID'])


20222502096

In [64]:
cursor.execute('SELECT student_id FROM students')
existing_student_ids = set(row[0] for row in cursor.fetchall())

for row in row_values:
    
    uni_id = int(row['University ID'])
    if uni_id in existing_student_ids:
        print(f'skipped {uni_id}')
        continue

    first_name = null_prevention(row['First Name'])
    
    last_name = null_prevention(row['Last Name'])
    
    email = null_prevention(row['University Email Address'])
    
    phone = null_prevention(row['Phone Number'])

    gender = null_prevention(row['Gender'])
    
    faculty = null_prevention(row['What is your faculty?'])
    
    major = null_prevention(row['What is your major? (eg. Computer Engineering)'])

    year_str = null_prevention(row['What year are you in?'])
    year = year_str[0]
    # print(f'{year = }')

    ai_familiarity = int(row['How familiar are you with AI'])
    
    hours_passed = int(row['Credit Hours Passed'])

    linkedin = null_prevention(row['LinkedIn (Optional)'])

    git_hub = null_prevention(row['GitHub (Optional)'])

    communities = null_prevention(row['Which community are you a part of? (Optional)'])
    
    AAAI_member = False
    IEEE_member = False

    if communities:
        if 'AAAI' in communities:
            AAAI_member = True
        if 'IEEE' in communities:
            IEEE_member = True

    interested_activities = null_prevention(row['Which activity are you likely to join'])
    
    remarks = ''

    if interested_activities:
        remarks = remarks + 'Interested in : ' + interested_activities
        

    sql = '''
    CALL insert_student_from_sheets(
    %s, %s, %s, %s, %s, 
    %s, %s, %s, %s, %s, 
    %s, %s, %s, %s, %s,
    %s
    )
    '''

    values = (
        uni_id,
        first_name,
        last_name,
        email,
        phone,
        gender,
        faculty,
        major,
        year,
        ai_familiarity,
        hours_passed,
        linkedin,
        git_hub,
        AAAI_member,
        IEEE_member,
        remarks
    )
    
    cursor.execute(sql, values)

db.commit()
print('done')

done


In [40]:
cursor.fetchall()

[('USER ALREADY IN DATABASE',)]

In [57]:
sql = '''
delete from students
'''

In [58]:
cursor.execute(sql)

In [65]:
sql = '''
select *
from students
'''

In [66]:
cursor.execute(sql)

In [67]:
s = set(row for row in cursor.fetchall())

In [68]:
s

{(20221502065,
  'Basel',
  'Al-Dwairi',
  0,
  'B.AlDwairi@gju.edu.jo',
  '962791993486',
  'Male',
  4,
  'School of Computing',
  'Computer Engineering',
  137,
  9,
  'https://github.com/Basel-Aldwairi',
  'https://www.linkedin.com/in/basel-al-dwairi/',
  None,
  0,
  1,
  'Interested in : Carnavals, Hackathons, Programming Competitions, Internships'),
 (20222502096,
  'Yazan',
  'Ayash',
  0,
  'y.ayash@gju.edu.jo',
  '962792040345',
  'Male',
  4,
  'School of Computing',
  'Computer Engineering',
  136,
  6,
  'https://github.com/NeatXD',
  'www.linkedin.com/in/yazan-ayash-140615244',
  None,
  1,
  0,
  'Interested in : Carnavals, Hackathons, Programming Competitions, Internships, Cycling Events, Seminars, Workshops')}

In [45]:
db.close()